In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import torch
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import fastai

from fastai.imports import *
from fastai.vision import *
from fastai.vision.all import *
from fastai.metrics import error_rate, accuracy

In [3]:
## Defining Directories
work_dir = Path('/kaggle/working/')
path = Path('../input/hotel-id-2021-fgvc8')
train = path/ '/train_images'
test =  path/'/test_images'

## csv
sample_sub = path/'sample_submission.csv'
labels = path/'train.csv'

In [4]:
train_data = pd.read_csv(labels)
train_data.head(5)

,image,chain,hotel_id,timestamp
0,8000d59f6a3d765f.jpg,85,24607,2018-11-28 05:42:31
1,8000febe81817f7f.jpg,6,25389,2017-01-01 07:01:46
2,80053befc2d42fba.jpg,3,672,2018-01-26 13:22:07
3,8007fa0a8fe89f9d.jpg,5,63866,2020-07-03 02:31:31
4,800a4a6b6f6d2df6.jpg,0,12036,2017-09-22 17:59:22


In [5]:
sample_sub = pd.read_csv(sample_sub)
sample_sub.head(5)

,image,hotel_id
0,99e91ad5f2870678.jpg,36363 53586 18807 64314 60181
1,b5cc62ab665591a9.jpg,36363 53586 18807 64314 60181
2,d5664a972d5a644b.jpg,36363 53586 18807 64314 60181


In [6]:
# !tar xzvf /kaggle/input/hotelid-224/kaggle_hotelid_train_imgs_224x224.tgz

In [7]:
train_dir= './kaggle/working/train/'
train_data['chain_image'] = train_dir + train_data['chain'].astype(str) + '/' + train_data['image'].astype(str)

train_data['hotel_id'] = train_data['hotel_id'].astype(str)
train_data[:3]


,image,chain,hotel_id,timestamp,chain_image
0,8000d59f6a3d765f.jpg,85,24607,2018-11-28 05:42:31,./kaggle/working/train/85/8000d59f6a3d765f.jpg
1,8000febe81817f7f.jpg,6,25389,2017-01-01 07:01:46,./kaggle/working/train/6/8000febe81817f7f.jpg
2,80053befc2d42fba.jpg,3,672,2018-01-26 13:22:07,./kaggle/working/train/3/80053befc2d42fba.jpg


In [8]:
train_subset = train_data.loc[train_data['chain']==90]
train_subset['hotel_id'] = train_subset['hotel_id'].astype(str)
train_subset[:3]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,image,chain,hotel_id,timestamp,chain_image
9,80196e6999ce63cf.jpg,90,55532,2020-06-16 02:46:59,./kaggle/working/train/90/80196e6999ce63cf.jpg
26,802b5ed622fd3587.jpg,90,35252,2020-02-04 02:05:29,./kaggle/working/train/90/802b5ed622fd3587.jpg
53,803be117a845fe9d.jpg,90,61942,2016-06-28 02:17:49,./kaggle/working/train/90/803be117a845fe9d.jpg


In [9]:
classes = train_subset.hotel_id.unique()
classes = list(classes)
len(classes)

270

In [10]:
# from distutils.dir_util import copy_tree
# fromDir ='../input/submission'
# toDir ='.'
# copy_tree(fromDir, toDir)

# sub = './submission.csv'
# df = pd.read_csv(sub)
# df.head(5)

In [11]:
# from distutils.dir_util import copy_tree
# fromDir ='../input/hotel-id-2021-fgvc8/test_images'
# toDir ='./kaggle/working/test_images'
# copy_tree(fromDir, toDir)

In [12]:
# print(train_data['hotel_id'].value_counts().sort_values(ascending=False))

In [13]:
batch_size = 32
def create_dataLoader(data_dir):
    data = ImageDataLoaders.from_folder(data_dir,
                                        train = 'train',
                                        test= 'test_images',
                                        valid_pct=0.2,
                                        item_tfms=RandomResizedCrop(128, min_scale=0.35),
                                        batch_tfms=aug_transforms(),
                                        normalize =imagenet_stats,
                                        bs=batch_size
                                       )
    return data

## Creating a data Loader of the entire Dataset

In [14]:

file_path='./input/hotel-id-2021-fgvc8/train_images/90'
data = create_dataLoader(work_dir)

TypeError: 'NoneType' object is not iterable

In [15]:
data.show_batch()

NameError: name 'data' is not defined

In [16]:
def training_models(data, model_name):
    lr = 1e-1    
    learn = cnn_learner(data,
                    models.resnet50,
                    metrics=[accuracy, error_rate], 
                    model_dir="."
                   )
    learn.fit_one_cycle(1, 1e-2, wd=0.01)
    
    learn.save(model_name)
    
    return learn

In [17]:
# model_name ='fa_allData_rn50'
from distutils.dir_util import copy_tree
fromDir ='../input/data-model/'
toDir ='.'
copy_tree(fromDir, toDir)
# learn = training_models(data, model_name)

['./submission.csv', './fa_allData_rn50.pth']

In [18]:
def load_model():
    path = '.'
    learn = load_learner(path, 'fa_allData_rn50.pth')
    return learn

## Predictions

In [19]:
test = '../input/hotel-id-2021-fgvc8/test_images/'
predict =[]
folder_list = os.listdir(test)
for i in range(len(folder_list)):
    test_img = test + folder_list[i]
    print(test_img)
    predict.append(learn.predict(test_img, with_input=False))

../input/hotel-id-2021-fgvc8/test_images/b5cc62ab665591a9.jpg


NameError: name 'learn' is not defined

In [20]:
sample_submission = pd.read_csv('../input/hotel-id-2021-fgvc8/sample_submission.csv')
sample_submission.head()
submission = sample_submission.copy()

In [21]:
submission = sample_submission.copy()
for i in range(len(submission)):
    pred =predict[i]
    submission.iloc[i, 1:] = pred[1].tolist()
submission.head()

IndexError: list index out of range

In [22]:
os.chdir(".")
submission.to_csv("submission.csv", index=False)

In [23]:
rm -r './kaggle/working/train'

rm: cannot remove './kaggle/working/train': No such file or directory
